In [ ]:
import os
import random
import pandas as pd
import pyarrow.parquet as pq
from datetime import datetime, timedelta
from typing import List, Dict

from plot_func import create_plotly_config

STORAGE_ROOT = "./storage"
SCORE_ROOT = os.path.join(STORAGE_ROOT, "score")

# [핵심] 메모리에 데이터를 유지할 전역 저장소
# 구조: { "session_id_또는_유저별키": DataFrame } 
# 간단하게 단일 유저라고 가정하고 하나의 변수에 저장합니다. (실제 서비스에선 Redis 등을 씀)
GLOBAL_DATA_CACHE = {
    "full_df": None,
    "aggregated_scores": None,
    "target_tool": None
}

# (parse_filename, load_tool_specific_scores 등 기존 함수 유지...)
# ...

def load_data_to_memory(process, tools, recipes, start_date_str, end_date_str, target_tool=None):
    """
    1단계: 파일을 모두 읽어서 메모리(GLOBAL_DATA_CACHE)에 저장합니다.
    """
    try:
        s_date = datetime.strptime(start_date_str, "%Y-%m-%d").date()
        e_date = datetime.strptime(end_date_str, "%Y-%m-%d").date()
    except ValueError: raise ValueError("Invalid date")

    # 1. 파일 찾기
    files_to_read = []
    for tool in tools:
        tool_dir = os.path.join(STORAGE_ROOT, process, tool)
        if not os.path.exists(tool_dir): continue
        for f in os.listdir(tool_dir):
            if f.endswith(".parquet"):
                f_tool, f_date, f_recipe = parse_filename(f)
                if f_tool and f_date and s_date <= f_date <= e_date and f_recipe in recipes:
                    files_to_read.append({"path": os.path.join(tool_dir, f), "tool": f_tool})

    if not files_to_read:
        GLOBAL_DATA_CACHE["full_df"] = None
        return False

    # 2. 데이터 읽기
    dfs = []
    for item in files_to_read:
        df = pd.read_parquet(item["path"])
        df["tool_name"] = item["tool"]
        dfs.append(df)
    
    full_df = pd.concat(dfs)

    # 3. 점수 데이터 로드
    tool_scores = load_tool_specific_scores(s_date, e_date, tools)

    # 4. 전역 변수에 저장 (캐싱)
    GLOBAL_DATA_CACHE["full_df"] = full_df
    GLOBAL_DATA_CACHE["tool_scores"] = tool_scores
    GLOBAL_DATA_CACHE["target_tool"] = target_tool
    
    return True

def build_lightweight_tree(issue_type="Particle"):
    """
    2단계: 메모리에 있는 데이터로 '껍데기' 트리만 만듭니다. (차트 X, 히스토리 X)
    """
    full_df = GLOBAL_DATA_CACHE["full_df"]
    tool_scores = GLOBAL_DATA_CACHE["tool_scores"]
    target_tool = GLOBAL_DATA_CACHE["target_tool"]

    if full_df is None: return None

    # 데이터 요약 (평균값, 점수만 계산)
    merged_rows = []
    
    # groupby로 요약 정보만 추출
    for (group, sensor), sub_df in full_df.groupby(['group', 'sensor']):
        
        # 툴별 마지막 값만 추출하여 평균 계산
        last_vals = []
        target_val = 0
        
        for tool_name, tool_df in sub_df.groupby('tool_name'):
            # history 전체를 가져오지 않고 마지막 값만 가져옴 (속도 향상)
            val = tool_df['value'].iloc[-1] if not tool_df.empty else 0
            last_vals.append(val)
            if tool_name == target_tool:
                target_val = val
        
        avg_val = sum(last_vals) / len(last_vals) if last_vals else 0
        
        # Diff 계산
        diff_val = 0
        if target_tool and target_val:
            others = [v for v in last_vals if v != target_val]
            if others:
                other_avg = sum(others) / len(others)
                diff_val = abs(target_val - other_avg)
        
        score_map = tool_scores.get(sensor, {})

        merged_rows.append({
            "group": group, 
            "sensor": sensor,
            "avg_value": avg_val,
            "diff_value": diff_val,
            "score_map": score_map
        })
    
    merged_df = pd.DataFrame(merged_rows)

    # 트리 구조 생성
    groups = merged_df.groupby('group')
    children_list = []

    for group_name, group_df in groups:
        sensors = []
        # 상위 50개 등 필터링이 필요하면 여기서 수행
        for _, row in group_df.iterrows():
            sensors.append({
                "name": row['sensor'],
                "value": row['avg_value'],
                "diff_value": row['diff_value'],
                "score_map": row['score_map'],
                "type": "sensor" 
                # chart_config 없음! 
            })
        
        children_list.append({ "name": group_name, "children": sensors, "type": "group" })

    root_name = f"{issue_type} Analysis"
    if target_tool: root_name += f" (Target: {target_tool})"

    return { "name": root_name, "children": children_list, "type": "root" }

def get_chart_data_from_memory(sensor_name):
    """
    3단계: 메모리에서 특정 센서의 데이터만 꺼내 차트 JSON을 만듭니다.
    """
    full_df = GLOBAL_DATA_CACHE["full_df"]
    target_tool = GLOBAL_DATA_CACHE["target_tool"]
    
    if full_df is None: return None

    # 메모리에 있는 DataFrame에서 해당 센서 데이터만 필터링 (매우 빠름)
    sensor_df = full_df[full_df['sensor'] == sensor_name]
    
    if sensor_df.empty: return None

    # 툴별 히스토리 추출
    tool_history_map = {}
    for tool_name, tool_df in sensor_df.groupby('tool_name'):
        hist = []
        for h in tool_df['history']:
            hist.extend([float(x) for x in h])
        tool_history_map[tool_name] = hist
        
    # 차트 설정 생성
    return create_plotly_config(sensor_name, tool_history_map, target_tool)

def get_related_sensors_from_memory(sensor_name):
    """
    (선택) 연관 센서 데이터도 클릭 시 가져오려면 유사하게 구현
    """
    # 현재는 전체 로직이 복잡해지므로 생략하거나, 
    # 위 get_chart_data_from_memory에 포함해서 리턴할 수 있습니다.
    pass

In [ ]:
# main.py

# ... import ...

@app.post("/api/load")
async def load_data_api(req: LoadRequest):
    try:
        # 1. 데이터 로드 (오래 걸림)
        success = data_load.load_data_to_memory(
            req.process, req.tools, req.recipes, req.start_date, req.end_date, req.target_tool
        )
        if not success:
            return JSONResponse({"error": "No data found"}, status_code=404)
        
        # 2. 가벼운 트리 생성 (빠름)
        tree_structure = data_load.build_lightweight_tree(req.issue_type)
        return JSONResponse(tree_structure)
        
    except Exception as e:
        return JSONResponse({"error": str(e)}, status_code=500)

class ChartRequest(BaseModel):
    sensor_name: str

@app.post("/api/chart")
async def get_chart_api(req: ChartRequest):
    # 메모리에서 즉시 꺼냄 (매우 빠름)
    chart_config = data_load.get_chart_data_from_memory(req.sensor_name)
    if not chart_config:
         return JSONResponse({"error": "Sensor not found"}, status_code=404)
    return JSONResponse(chart_config)

In [ ]:
// script.js

// ... loadData ...
// (기존과 동일하게 /api/load 호출하여 treeData 받아서 renderTree 호출)

function renderTree(treeData, targetTool) {
    // ... D3 설정 동일 ...

    node.append("circle").attr("r", 8)
        .on("click", async function(event, d) {  // async 필수
            d3.selectAll("circle").classed("selected", false);
            d3.select(this).classed("selected", true);
            // ... 링크 하이라이트 ...

            // [변경] 차트 데이터가 없으면 서버에 요청 (메모리에서 가져옴)
            if (!d.data.chart_config) {
                document.getElementById("placeholder").innerText = "Fetching Chart...";
                document.getElementById("placeholder").style.display = "block";
                Plotly.purge('plotly-chart');

                try {
                    const res = await fetch("/api/chart", {
                        method: "POST", headers: {"Content-Type": "application/json"},
                        body: JSON.stringify({ sensor_name: d.data.name })
                    });
                    
                    if(res.ok) {
                        const chartData = await res.json();
                        d.data.chart_config = chartData; // 가져온 데이터 저장 (캐싱)
                    }
                } catch(e) { console.error(e); }
            }

            // 차트 그리기
            if (d.data.chart_config) {
                document.getElementById("placeholder").style.display = "none";
                Plotly.newPlot('plotly-chart', d.data.chart_config.data, d.data.chart_config.layout, {responsive: true});
            }

            // ... 리스트 표시 로직 (만약 리스트 데이터도 무거우면 위와 똑같이 따로 요청) ...
            
            event.stopPropagation();
        });
    // ...
}

In [2]:
import pandas as pd
import json

# 1. 임의의 평평한 데이터 생성 (엑셀에서 읽었다고 가정)
raw_data = [
    {'group': 'Temp', 'sensor': 'Top_Temp', 'value': 100},
    {'group': 'Temp', 'sensor': 'Bottom_Temp', 'value': 95},
    {'group': 'Temp', 'sensor': 'Wall_Temp', 'value': 98},
    {'group': 'Gas',  'sensor': 'H2_Flow', 'value': 50},
    {'group': 'Gas',  'sensor': 'N2_Flow', 'value': 200},
    {'group': 'Press', 'sensor': 'Chamber_Press', 'value': 760},
]

df = pd.DataFrame(raw_data)

# (1) Group 별로 묶기
groups = df.groupby('group')
children_list = []

for group_name, group_df in groups:
    # group_name: "Temp"
    
    # (2) 해당 그룹 안의 센서들을 리스트로 만듦
    sensors = []
    for _, row in group_df.iterrows():
        sensors.append({
            "name": row['sensor'], # D3에 보여줄 이름
            "value": row['value'], # 기타 데이터
            "type": "sensor"       # 내가 구분하기 위한 태그
        })
    
    # (3) 그룹 노드를 만들고 센서들을 자식(children)으로 넣음
    children_list.append({
        "name": group_name,    # 그룹 이름 ("Temp")
        "children": sensors,   # 자식 노드들 (센서 리스트)
        "type": "group"
    })

# (4) 최상위 루트 노드 생성
tree_structure = {
    "name": "Equipment 1",
    "children": children_list,
    "type": "root"
}

tree_structure 

{'name': 'Equipment 1',
 'children': [{'name': 'Gas',
   'children': [{'name': 'H2_Flow', 'value': 50, 'type': 'sensor'},
    {'name': 'N2_Flow', 'value': 200, 'type': 'sensor'}],
   'type': 'group'},
  {'name': 'Press',
   'children': [{'name': 'Chamber_Press', 'value': 760, 'type': 'sensor'}],
   'type': 'group'},
  {'name': 'Temp',
   'children': [{'name': 'Top_Temp', 'value': 100, 'type': 'sensor'},
    {'name': 'Bottom_Temp', 'value': 95, 'type': 'sensor'},
    {'name': 'Wall_Temp', 'value': 98, 'type': 'sensor'}],
   'type': 'group'}],
 'type': 'root'}

In [ ]:
/* static/css/style.css */

/* 기존 Body, Sidebar 등은 유지하되 .content 내부 레이아웃 변경 */
body {
    font-family: 'Segoe UI', sans-serif;
    background-color: #f4f6f9;
    margin: 0; padding: 0;
    display: flex; height: 100vh; overflow: hidden;
}

/* Sidebar 스타일 유지 */
.sidebar {
    width: 270px; min-width: 270px;
    background: #fff; border-right: 1px solid #ddd;
    padding: 20px; display: flex; flex-direction: column; gap: 12px;
    box-shadow: 2px 0 5px rgba(0,0,0,0.05); z-index: 10;
    overflow-y: auto;
}
/* ... Sidebar 내부 요소 스타일 유지 ... */

/* [핵심 변경] Content: 가로 배치 (좌측 차트 / 우측 트리+KG) */
.content {
    flex: 1;
    display: flex;
    flex-direction: row; /* 가로 배치 */
    padding: 15px;
    gap: 15px;
    overflow: hidden;
}

/* [신규] 좌측 패널 (메인 차트) - 화면의 45% */
.left-panel {
    flex: 0.9; 
    background: white;
    border-radius: 10px;
    border: 1px solid #e0e0e0;
    display: flex;
    flex-direction: column;
    padding: 10px;
    min-width: 0; /* Flexbox 내 차트 리사이징 이슈 방지 */
}

/* [신규] 우측 패널 (트리 + KG) - 화면의 55% */
.right-panel {
    flex: 1.1;
    display: flex;
    flex-direction: column; /* 세로 배치 */
    gap: 15px;
    min-width: 0;
}

/* 우측 상단: 트리 + 리스트 */
.right-top {
    flex: 1;
    display: flex;
    gap: 10px;
    min-height: 0;
}

/* 우측 하단: Knowledge Graph (Scatter Plot) */
.right-bottom {
    flex: 1;
    background: white;
    border-radius: 10px;
    border: 1px solid #e0e0e0;
    position: relative;
    min-height: 0;
    display: flex;
    flex-direction: column;
}

/* 내부 요소들 */
#tree-wrapper { flex: 2; background: white; border-radius: 10px; border: 1px solid #e0e0e0; position: relative; }
#list-wrapper { flex: 1; min-width: 180px; background: white; border-radius: 10px; border: 1px solid #e0e0e0; display: flex; flex-direction: column; overflow: hidden; }
#kg-wrapper { flex: 1; width: 100%; height: 100%; position: relative; }

/* ... D3, Plotly 관련 스타일 유지 ... */
.link-structure { stroke: #999; stroke-opacity: 0.3; stroke-width: 1px; }
.link-correlation { stroke: #ff4444; stroke-opacity: 0.8; stroke-width: 2px; stroke-dasharray: 5,5; }
.node-group { fill: #FFD700; stroke: #fff; stroke-width: 2px; }
.node-sensor { fill: #87CEEB; stroke: #fff; stroke-width: 1.5px; }
.edge-label { font-family: Arial, sans-serif; font-size: 9px; fill: #d32f2f; font-weight: bold; pointer-events: none; text-anchor: middle; text-shadow: -1px -1px 0 #fff, 1px -1px 0 #fff; }
.value-label-bg { fill: #2e7d32; opacity: 0.9; }
.value-label-text { font-size: 10px; fill: white; font-weight: bold; pointer-events: none; }

In [ ]:
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <title>Integrated Analysis</title>
    <script src="https://d3js.org/d3.v7.min.js"></script>
    <script src="https://cdn.plot.ly/plotly-2.27.0.min.js"></script>
    <link rel="stylesheet" href="/static/css/style.css">
</head>
<body>

<div class="sidebar">
    <h2>Data Load Options</h2>
    
    <div class="control-group">
        <label>1. Process</label>
        <select id="process-select" onchange="updateToolList()">
            <option value="" disabled selected>Select Process</option>
            <option value="process1">Process 1</option>
            <option value="process2">Process 2</option>
            <option value="process3">Process 3</option>
        </select>
    </div>

    <div class="control-group">
        <label>2. Date Range</label>
        <div style="display:flex; gap:5px;">
            <input type="date" id="start-date" value="2025-12-01" style="flex:1;">
            <input type="date" id="end-date" value="2025-12-10" style="flex:1;">
        </div>
    </div>

    <div class="control-group">
        <label>3. Tools (Multi-select)</label>
        <!-- onchange 제거: 자동 업데이트 방지 -->
        <select id="tool-select" multiple onchange="updateTargetToolOptions()">
            <option disabled>Wait for process</option>
        </select>
    </div>

    <!-- [추가] 레시피 로드 버튼 -->
    <button onclick="updateRecipeList()" style="margin-bottom: 10px; background:#e0e0e0; border:1px solid #ccc; padding:5px; cursor:pointer;">
        Get Recipe from Select Option
    </button>

    <div class="control-group">
        <label>4. Recipes (Multi-select)</label>
        <select id="recipe-select" multiple>
            <option disabled>Click 'Get Recipe' button</option>
        </select>
    </div>

    <div class="control-group" style="padding: 10px; background: #ffebee; border-radius: 5px; border: 1px solid #ffcdd2;">
        <label style="color: #d32f2f;">★ Target Tool</label>
        <select id="target-tool-select">
            <option disabled selected>Select tools above</option>
        </select>
    </div>

    <button class="btn-primary" onclick="loadData()">Load Data (File I/O)</button>

    <hr style="width:100%; margin: 15px 0;">

    <!-- [변경] 데이터 로드 후 변경하는 View 옵션 -->
    <h3>View Options</h3>
    <div class="control-group">
        <label>Issue Type (Refresh Tree)</label>
        <!-- 데이터 로드 없이 트리만 갱신 -->
        <select id="issue-select" onchange="refreshView()">
            <option value="Particle">Particle</option>
            <option value="Uniformity">Uniformity</option>
            <option value="Concentration">Concentration</option>
        </select>
    </div>
    <div class="control-group">
        <label>KG Threshold</label>
        <input type="range" id="kg-threshold" min="0" max="1" step="0.1" value="0.5" 
               oninput="document.getElementById('th-val').innerText=this.value" onchange="drawKG()">
        <span id="th-val" style="text-align:right; font-size:0.8em;">0.5</span>
    </div>
</div>

<div class="content">
    
    <!-- 좌측: 상세 차트 -->
    <div class="left-panel">
        <h4 style="margin:0 0 10px 0; color:#555;">Detailed Chart</h4>
        <div id="plotly-chart" style="width:100%; height:100%;"></div>
        <div id="placeholder" style="color:#999; text-align:center; margin-top:50%;">Select a node to view chart</div>
    </div>

    <!-- 우측 패널 -->
    <div class="right-panel">
        
        <!-- 우측 상단: 트리 + 리스트 -->
        <div class="right-top">
            <div id="tree-wrapper"></div>
            <div id="list-wrapper">
                <div id="list-header">Related Sensors</div>
                <div id="list-content">
                    <div style="color:#999; text-align:center; margin-top:20px;">Select a node</div>
                </div>
            </div>
        </div>

        <!-- 우측 하단: KG (Scatter Plot) -->
        <div class="right-bottom">
            <div style="padding:5px 10px; background:#fafafa; border-bottom:1px solid #eee; font-size:0.9rem; font-weight:bold;">
                Knowledge Graph (Scatter)
            </div>
            <div id="kg-wrapper"></div>
        </div>
    </div>

</div>

<script src="/static/js/script.js"></script>

</body>
</html>